# Clustering Crypto

In [68]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [69]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv", index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [70]:
crypto_df.shape

(1252, 6)

In [71]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [72]:
# Keep all the cryptocurrencies that are being traded.
crypto_trading = crypto_df[(crypto_df["IsTrading"] == True)]
crypto_trading.shape

(1144, 6)

In [73]:
crypto_trading["Algorithm"].isnull().unique()

array([False])

In [74]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_trading.shape

(1144, 6)

In [75]:
# Remove the "IsTrading" column. 
crypto_trading = crypto_trading.drop("IsTrading", axis=1)
crypto_trading.head()
crypto_trading.shape

(1144, 5)

In [76]:
crypto_trading = crypto_trading.dropna(how="any")

In [77]:
# Remove rows that have at least 1 null value.
crypto_trading.shape

(685, 5)

In [82]:
crypto_trading[crypto_trading["TotalCoinsMined"] == 0]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
808,808,SHA-256,PoW/PoS,0.0,0
XBS,Bitstake,X11,PoW/PoS,0.0,1300000
ACOIN,ACoin,SHA-256,PoW,0.0,1600000
AERO,Aero Coin,X13,PoS,0.0,7000000
APEX,ApexCoin,X13,PoW/PoS,0.0,6000000
...,...,...,...,...,...
QCN,Quazar Coin,CryptoNight,PoW,0.0,18446744
PKB,ParkByte,SHA-256,PoW/PoS,0.0,25000000
DOT,Dotcoin,Scrypt,PoW,0.0,890000000
THC,The Hempcoin,Scrypt,PoW/PoS,0.0,300000000


In [83]:
# Keep the rows where coins are mined.
crypto_trading = crypto_trading[crypto_trading["TotalCoinsMined"] != 0]
crypto_trading.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [84]:
crypto_trading["TotalCoinsMined"].unique()

array([ 4.19999538e+01,  1.05518490e+09,  2.92794246e+10,  1.79271750e+07,
        1.07684223e+08,  6.30392433e+07,  9.03129438e+06,  1.72011431e+07,
        1.13359703e+08,  7.38305625e+06,  2.74157000e+09,  1.14062191e+10,
        1.28886200e+06,  1.19953349e+07,  5.76831000e+06,  2.55425356e+07,
        1.11497340e+07,  1.22057954e+07,  1.81351000e+07,  6.35423900e+08,
        1.68825425e+09,  8.80000000e+01,  1.70204390e+08,  1.28940951e+08,
        7.35779479e+06,  1.23062801e+08,  2.59557165e+08,  4.95485851e+07,
        3.34429883e+07,  4.25794769e+04,  7.21995665e+06,  1.03911665e+06,
        4.70387952e+06,  4.99698621e+06,  3.99999973e+10,  3.30448775e+06,
        1.59295307e+10,  1.87738575e+10,  1.25390152e+10,  2.09104788e+07,
        2.18871896e+08,  1.94967863e+07,  5.67970500e+06,  4.61829905e+08,
        1.14273215e+06,  2.08669093e+08,  8.84125802e+08,  6.56727205e+07,
        4.16586480e+07,  8.92669337e+07,  1.67852009e+09,  9.05957530e+10,
        1.76509140e+07,  

In [85]:
# Keep the rows where coins are mined.
crypto_trading.shape

(533, 5)

In [86]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name = crypto_trading["CoinName"]
coin_df = pd.DataFrame(coin_name)
coin_df.shape

(533, 1)

In [87]:
# view df 
coin_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [88]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_trading_2 = crypto_trading.drop("CoinName", axis=1)
crypto_trading_2.shape


(533, 4)

In [89]:
crypto_trading_2.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [90]:
crypto_trading_2.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [91]:
crypto_trading_2["TotalCoinSupply"] = crypto_trading_2["TotalCoinSupply"].astype(float)

In [92]:
crypto_trading_2.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [93]:
# Use get_dummies() to create variables for text features.
crypto_encoded = pd.get_dummies(crypto_trading_2, columns=["Algorithm", "ProofType"])
crypto_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
#see shape of get dummies df
crypto_encoded.shape

(533, 100)

In [95]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(crypto_encoded)
crypto_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [96]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.33098013,  0.96034637, -0.6234373 ],
       [-0.31425099,  0.96042742, -0.6238748 ],
       [ 2.31857706,  1.73982421, -0.78877363],
       ...,
       [ 0.32589167, -2.26566592,  0.45003753],
       [-0.15171464, -1.91841817,  0.57207937],
       [-0.28043159,  0.63207764, -0.27421609]])

In [97]:
# Create a DataFrame with the three principal components.
crypto_pca_df = pd.DataFrame(crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_trading_2.index)
crypto_pca_df.head()

,PC 1,PC 2,PC 3
42,-0.330980,0.960346,-0.623437
404,-0.314251,0.960427,-0.623875
1337,2.318577,1.739824,-0.788774
BTC,-0.150608,-1.214870,0.180136
ETH,-0.159895,-2.027589,0.455874


In [98]:
crypto_pca_df.shape

(533, 3)

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [99]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1,11))


In [100]:
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)

In [101]:
elbow_data = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.head(10)

,k,inertia
0,1,3646.014475
1,2,2664.797164
2,3,1494.990218
3,4,560.047792
4,5,370.392744
5,6,289.915192
6,7,230.126842
7,8,182.885936
8,9,153.579513
9,10,122.368183


In [102]:
elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks = k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [103]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)

In [104]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_trading_2, crypto_pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = coin_df
clustered_df.head()

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.330980,0.960346,-0.623437,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.314251,0.960427,-0.623875,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.318577,1.739824,-0.788774,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.150608,-1.214870,0.180136,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.159895,-2.027589,0.455874,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.162018,-1.205337,0.016858,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.394038,1.305263,-0.576235,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.148117,-2.137279,0.404248,Monero,1
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.158330,-2.027696,0.455856,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.151714,-1.918418,0.572079,ZCash,1


In [105]:
clustered_df["CoinName"]

42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 533, dtype: object

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [129]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", hover_data=["Algorithm"], hover_name="CoinName")
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [107]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "CoinName", "Class"],sortable=True, selectable=True)


:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,Class]

In [108]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df["CoinName"].unique()))

532


In [131]:
coin_array = clustered_df[['TotalCoinsMined', 'TotalCoinSupply']].to_numpy()

In [132]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()
scaled_coin = scaler.fit_transform(coin_array, y=None)


In [133]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_coin




array([[5.94230127e-03, 4.20000000e-11],
       [7.00182308e-03, 5.32000000e-04],
       [3.53420682e-02, 3.14159265e-01],
       ...,
       [6.92655266e-03, 1.40022261e-03],
       [5.94962775e-03, 2.10000000e-05],
       [5.94243008e-03, 1.00000000e-06]])

In [137]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_clustered_df = pd.DataFrame(scaled_coin, columns=['TotalCoinsMined', 'TotalCoinSupply'], index=clustered_df.index)
scaled_clustered_df.head()

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_clustered_df["CoinName"] = clustered_df["CoinName"]
scaled_clustered_df.head()


# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_clustered_df["Class"] = clustered_df["Class"]
scaled_clustered_df.head()



,TotalCoinsMined,TotalCoinSupply,CoinName,Class
42,0.005942,4.200000e-11,42 Coin,0
404,0.007002,5.320000e-04,404Coin,0
1337,0.035342,3.141593e-01,EliteCoin,0
BTC,0.005960,2.100000e-05,Bitcoin,1
ETH,0.006050,0.000000e+00,Ethereum,1


In [139]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_clustered_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_name="CoinName")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)